# Modele de réseau neuronale

Toutes les étapes pour arriver à un modele correct

## Imports

In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
#import seaborn as sns #On ne sait jamais que ça serve
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler #Normalisation #Il faut l'installer aussi avec conda install
import random


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import  Dense, Conv3D, BatchNormalization,MaxPooling3D, Dropout, LSTM
from tensorflow.keras.utils import to_categorical

#conda install pydot
#conda install pydotplus
#conda install graphviz
from tensorflow.keras.utils import plot_model
#import pydot


import threading
from queue import Queue, Empty
from time import sleep, time

In C:\Users\nicol\anaconda3\envs\DL3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\nicol\anaconda3\envs\DL3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\nicol\anaconda3\envs\DL3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\nicol\anaconda3\envs\DL3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\nicol\anaconda3\envs\DL3\lib\site-packages\m

### Paramètres généraux

In [2]:
nb_classes = 10 #Nombre de classes
fps = 1
size = (40,30)
#Parametres du generator
pack_size = 50
batch_size = 25
epochs = 10
learning_rate = 0.05

In [3]:
#Verification qu'un modele avec ces paramètres n'existe pas déjà
folder = 'Saved_model'
modele_type = 'model_LSTM'
param = f"_{fps}_{size[1]}_{size[0]}_{nb_classes}_{epochs}_{batch_size}_{pack_size}_{int(learning_rate*1000)}mili"
full_name = folder + '\\' + modele_type + param
print(full_name)

Saved_model\model_LSTM_8_30_40_10_10_25_50_50mili


In [4]:
try:
    model_saved = keras.models.load_model(full_name)
    print("Pas besoin d'aller plus loin, le modele a déjà été fabriqué")
    model_charge = True
except:
    print("Modele non testé")
    model_charge = False

Modele non testé


## Fonctions d'imports et preprocess

#### Import

In [5]:
def get_mov_imgs_from_path(path,fps = -1,color = 'gray'):
    '''
    Retourne une liste d'images pour une vidéo. La liste d'image a le nombre de fps voulu
    reprend que le mouvement
    '''
    
    cap = cv2.VideoCapture(path)
    fps_actu = cap.get(cv2.CAP_PROP_FPS)
#     print(f"Traitement de la video n° {path}       ",end="\r")
    if fps <= -1: fps = fps_actu #Je peux ne pas donner de fps et ça va prendre le nombre d'fps initial
    ecart_voulu = int(1000/fps)
    ecart_initial = int(1000/fps_actu)
    imgs = []
    
    ret, frame = cap.read()
    
    while(cap.isOpened()):
        prev = frame
        ret, frame = cap.read()

        if ret: #Sinon ça plante quand il n'y a plus d'images
            #Récupère seulement certaines images
            t_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
            modulo = t_ms % ecart_voulu
            if modulo < ecart_initial:
                #Isoler les images
                
                diff = cv2.absdiff(frame,prev)  
                if color == 'gray':
                    diff_gray = cv2.cvtColor(diff,cv2.COLOR_BGR2GRAY)
                elif color == 'rgb':
                    diff_gray = diff

                    
                #Première normalisation (elle n'est peut-être pas obligatoire)
                max_ = np.max(diff_gray)
                if max_ == 0:
                    max_ = 1
                ratio = 255.0 / max_
                diff_gray = diff_gray * ratio
                
                
                #On va travailler avec des images en nuance de gris, c'est bcp plus simple
                imgs.append(diff_gray)
            
        else: #Va jusqu'au bout de la vidéo
            break
    else:
        print("Le fichier n'a pas pu être ouvert")
    cap.release()
    
    return imgs

#### Resize

In [6]:
def resize_imgs(imgs,nsize):
    '''
    Change la taille de l'image.
    Le premier élément de nsize est la longueur (width), le deuxième la hauteur (height)
    '''
    return [cv2.resize(img, dsize=nsize, interpolation=cv2.INTER_CUBIC) for img in imgs]

#### Normalisation

In [7]:
def scale_by_pixels(img, x_min, x_max):
    '''
    Scale une image entre 0 et 1.
    Méthode pas efficace mais fonctionnelle contrairement à d'autres méthodes 
    qui faisaient lignes par lignes ce qui causait des erreurs 
    dans le résultat (lignes dans l'image)
    '''
    new_img = img.copy()
    p_min = 1000
    p_max = -1000
    for line in img:
        for pixel in line:
            p_min = min(p_min,pixel)
            p_max = max(p_max,pixel)
    
    #p_min et p_max sont les min et max totaux de mon image
    for l, line in enumerate(img):
        for p,pixel in enumerate(line):
            nom = (pixel - p_min)*(x_max - x_min)
            denom = (p_max - p_min)
            if denom == 0: denom = 1
            new_img[l][p] = x_min + nom/denom
    return new_img

In [8]:
def normalize_imgs(imgs):
    '''
    Normalise une série d'images
    '''
    norm_imgs = []
    for img in imgs:
        norm_img = scale_by_pixels(img,0,1)
        norm_imgs.append(norm_img)
#     print(f"Après normalisation, max = {np.max(norm_imgs)} et min = {np.min(norm_imgs)}")
    return norm_imgs

## Fonction de générateur

In [9]:
def my_gen(labels_list, folder_path, pack_size = 10, pointer = 0, fps = 5, size = (160,120)):
    '''
    à partir de la liste des labels et de la position du foler
    Retourne:
    -une liste d'images de vidéos de shape (pack_size,n_frames,height,width[,channels])
        n_frames = fps*2.4
    -Une liste de numérique avec les labels correspondant aux images (de len = batch_size)
    -Un pointer qui permet de faire tourner my_gen à nouveau et recevoir les éléments suivants de la liste
    
    
    Normalement on peut envoyer X et y_num dans le modele directement (avec fit ou train_on_batch, les deux devraient fonctionner)
    Je vais essayer de travailler avec des thread pour avoir un thread en Train et un trhead en gen
    '''
    
    y_num = []
    X = []
    for i in range(pointer, pack_size + pointer):
        pointer = i+1
        if i >= len(labels_list):
            break
        label, video_name = labels_list[i]
        
        #Label en numérique
        y_num.append(labels_n[label])
        
        #Preprocess d'images
        file_path = folder_path + "\\" + video_name
        imgs_of_video = get_mov_imgs_from_path(file_path,fps,'gray') #Gray ou rgb
        resized_imgs = resize_imgs(imgs_of_video, size)
        norm_imgs = normalize_imgs(resized_imgs)
        
        X.append(norm_imgs)
        
    print() #Pour ne pas écrire sur la même ligne qu'avant
    X = np.array(X)
    y = np.array(y_num)
    
    #Ajouter une dimsension en à la fin est nécessaire pour certains modèle
    #Commenter cette ligne si ce n'est pas nécessaire
    X = np.expand_dims(X, axis=len(X.shape)) 
    
    # Si le modele est compile avec :loss='sparse_categorical_crossentropy'
    # Il faut un y catégorique , faire ça hors de la fonction
    #y = to_categorical(y, num_classes = nb_classes)

    return (X, y, pointer)

In [10]:
def gen_thread(labels_list, folder_path, pack_size = 10, fps = 5, size = (160,120)):
    '''
    S'assure que my_gen tourne toujours en parallèle du fit. Il prépare toujours 2 paquets de vidéo en avance
    '''
    pointer = 0
    while pointer < len(labels_list):
        X,y,pointer = my_gen(labels_list, folder_path, pack_size, pointer, fps, size)
        q.put((X,y))
        print(f"Pack of {pack_size} videos put in queue")

## Fabrication des modeles

#### Imports

In [11]:
def create_model(n_frames = 24, height = 120, width = 160, channels = 1, nb_classes = 10):
    #Gestion du nombre de frames
    #entre 12 et 24 frames, ça donne entre 3 et 6 images au LSTM, c'est très bien. 
    div_pool1 = 2
    div_pool2 = 2
    if n_frames < 12:
        div_pool1 = 1
    elif n_frames >= 36:
        div_pool1 = 3
    if n_frames < 6:
        div_pool2 = 1
    
    
    #Dernière couche avant reshape
    last_channel = 10
    
    #Gestion de la hauteur et la width:
    output_shape = int(height/27) * int(width/27) * last_channel
    
    #Divisé par 27 parce qu'on divise par 3, 3 fois (avec Maxpooling)

    
    sample_shape = (n_frames,height,width,channels) #width = 160, height = 120, nframes = 24, 3 channels si on est en RGB (si on est en gris on sait pas)
    
    model = Sequential()
    
    #model.add(Conv3D(128, strides =(1,1,1), padding="same", kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape,data_format='channels_first'))
    model.add(Conv3D(192, strides =(1,1,1), padding="same", kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape,data_format='channels_last'))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization(center=True, scale=True))
    model.add(Conv3D(96, strides=(1,1,1), padding="same", kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization(center=True, scale=True))
    model.add(MaxPooling3D(pool_size=(div_pool1, 3, 3)))
    model.add(Dropout(0.2))

    model.add(Conv3D(96, strides=(1,1,1), padding="same", kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(96, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization(center=True, scale=True))
    model.add(Conv3D(64, strides=(1,1,1), padding="same", kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization(center=True, scale=True))
    model.add(MaxPooling3D(pool_size=(div_pool2, 3, 3)))
    model.add(Dropout(0.2))

    model.add(Conv3D(48, strides=(1,1,1), padding="same", kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(48, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization(center=True, scale=True))
    model.add(Conv3D(32, strides=(1,1,1), padding="same", kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization(center=True, scale=True))
    model.add(MaxPooling3D(pool_size=(3, 3, 3), data_format = 'channels_first')) #Je ne veux pas diviser le nombre d'images cette fois-ci
    model.add(Dropout(0.2))

    model.add(Conv3D(last_channel, strides=(1,1,1), padding="same", kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(last_channel, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization(center=True, scale=True))


    model.add(layers.Reshape((int(n_frames/(div_pool1 * div_pool2)),output_shape)))


    #LSTM
    
    model.add(LSTM(units = 64,activation="tanh", recurrent_activation="sigmoid",return_sequences = False))


    #softmax
    model.add(Dense(nb_classes, activation='softmax'))

    return model

In [12]:
n_frames = int(fps*2.4)

if model_charge:
    print('Modele chargé')
    model = model_saved
else:
    print("Pas de modele chargé, il faut en créer un nouveau")
    model = create_model(n_frames = n_frames, height = size[1], width = size[0], nb_classes = nb_classes)

Pas de modele chargé, il faut en créer un nouveau


In [13]:
# model.summary()

In [14]:
plot_name = full_name + '_plot.png'
# plot_model(model, to_file = plot_name, show_shapes=True, show_layer_names=True)

In [15]:
#Compilation
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
              loss = tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

##### Chargement d'un modele préentrainé (par nous même)

In [16]:
#model_stolen_saved = keras.models.load_model('Saved_model\\modele_stolen_compile')

## Generator

In [17]:
def homemade_fit_gen_thread(model, labels_val,pack_size,epochs,batch_size,fps = 5,size = (160,120)):
    '''
    Charge les données à partir du csv labels_val et train le model avec
    les différents paramètres
    Le model est train.
    
    Si besoin, return de X,y et model (mais normalement le model original est train)
    '''
#     q = Queue()
    
    histories = []
    
    t = threading.Thread(target = gen_thread, daemon = True,#Daemon afin que le thread s'arrête avec le thread principal
                        args = ((labels_val, "DATA\\Videos",pack_size,fps, size))) 
    t.start()
    while True:
        X, y_num = q.get()
        print("I got the pack! I'm going to fit it")
        
        #Catégorisation pour train
        y = to_categorical(y_num, num_classes = nb_classes)
        
        history = model.fit(X, y,
                batch_size = 5,
                epochs=epochs,
                verbose=1,
                validation_split=0.2)
        histories.append(history)
        
        q.task_done() #Jsp si c'est utile
        
        #Il faut que le fit prenne plus de temps que le chargement des vidéos, sinon ça va pas aller
        
        if q.empty():
            print("J'ai peut-être fini, j'attends encore qq sec pour être sur d'avoir bien fini")
            sleep(60) #Je m'assure que c'est vraiment bien fini
            if q.empty():
                print("J'ai vraiment fini")
                break
            else:
                print("Je n'avais pas fini en fait")
    
    t.join()
    return model, histories, X, y

In [18]:
def homemade_fit_gen(model, labels_val,pack_size,epochs,batch_size,fps = 5,size = (160,120)):
    '''
    Charge les données à partir du csv labels_val et train le model avec
    les différents paramètres
    Le model est train.
    
    my_gen et fit tournent en parallèle grâce à un thread
    Si besoin, return de X,y et model (mais normalement le model original est train)
    '''
    
    histories = []
    
    pointer = 0
    can_continue = True
    
    while can_continue:
        X, y_num, pointer = my_gen(labels_val, "DATA\\Videos",pack_size = pack_size,fps = fps, size = size, pointer = pointer)
        
        #Catégorisation pour train
        y = to_categorical(y_num, num_classes = nb_classes)
        
        if pointer >= len(labels_val):
            can_continue = False
        print()

        #print('vidéos chargées, shape:',X.shape,'. y len : ',len(y_num)) #DEBUG
        with tf.device('/cpu:0'):
            history = model.fit(X, y,
                    batch_size = batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2)
        del X
        del y
        del y_num
        histories.append(history)
    return model, histories

## Import des données et traitement

In [19]:
#Import des labels
labels_csv = pd.read_csv("DATA\\labels.csv")
labels_name = pd.read_csv("DATA\\labels_uses.csv")
labels_tests_csv = pd.read_csv("DATA\\labels_tests.csv")

In [20]:
#Créé un dictionnaire qui associe le nom du label et un numérique afin de le mettre dans le modele
labels_n = {}
for i,label in enumerate(labels_name.values):
    labels_n[label[0]] = i
    labels_n[i] = label[0]

    
#Trier les labels pour ne prendre que ceux qui correspondent à labels_uses
labels = []
for label, video_name in labels_csv.values:
    if label in labels_n:
        labels.append((label,video_name))
        
#Idem pour le label de tests
labels_tests = []
for label, video_name in labels_tests_csv.values:
    if label in labels_n:
        labels_tests.append((label,video_name))


#Melange la liste de train
random.shuffle(labels) #Il faut absolument shuffle pour train



In [21]:
#Count de chaque label
labels_c = {}
for label in labels_name.values:
    labels_c[label[0]] = 0

for label,video_name in labels:
    if label in labels_c:
        labels_c[label] += 1

print(labels_c)

{'Ferme le poing': 53, 'Ouvre le poing': 56, 'Clap': 51, 'Swipe gauche main droite': 45, 'Swipe droit main gauche': 46, 'Doigts croises paumes': 47, 'Explosion avec les doigts ouverts': 49, 'Ouvrir un livre avec le poing ferme main D': 40, 'Ouvrir un livre avec le poing ferme main G': 35, 'Faire coucou': 62}


## Train

In [22]:
q = Queue(maxsize = 3) #Variable globale nécessaire pour le bon fonctionnement du gen en thread

In [ ]:
if model_charge:
    print('Il ne faut pas train le modele car il a déjà été chargé')
else:
    print('Il faut entrainer le modele')
    start = time()
    model, histories, _,_ = homemade_fit_gen(model, labels, pack_size = pack_size, epochs = epochs,batch_size = batch_size, fps = fps, size = size)
    end = time()
    print(f"ça a prit {end-start} secondes d'entrainer le modele")

Il faut entrainer le modele


Epoch 1/10
2/2 [==============================] - 52s 19s/step - loss: 2.3135 - accuracy: 0.0300 - val_loss: 3.0995 - val_accuracy: 0.1000
Epoch 2/10
2/2 [==============================] - 47s 19s/step - loss: 2.2925 - accuracy: 0.2433 - val_loss: 4.1388 - val_accuracy: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 47s 18s/step - loss: 2.2409 - accuracy: 0.2300 - val_loss: 3.2296 - val_accuracy: 0.0000e+00
Epoch 4/10
2/2 [==============================] - 47s 18s/step - loss: 2.0167 - accuracy: 0.2733 - val_loss: 4.7993 - val_accuracy: 0.0000e+00
Epoch 5/10


## Tests

In [ ]:
X_test, y_test, _ = my_gen(labels_tests, "DATA\\V_tests",pack_size = 1000,fps = fps, size = size)
#Grand batch_size afin de prendre toutes les données

In [ ]:
y_pred = model.predict(X_test)

### visualisation des  résultats

In [ ]:
import fidle.pwk as pwk

In [ ]:
y_pred_arg = np.argmax(y_pred, axis=-1)
plot_confusion_name = full_name + '_confusion.png'
pwk.plot_confusion_matrix(y_test,y_pred_arg,range(nb_classes),normalize=True,save_as = plot_confusion_name)

In [ ]:
print(labels_n)

In [ ]:
#Je peux afficher l'historique de chaque étape (je n'ai aps réussi à les faire fusionner)
#On ne peut afficher que si on a train

#history = histories[-1]
#pwk.plot_history(history, figsize=(6,4), save_as='03-history')

## Enregistrer le modèle

In [ ]:
#Le nom est fabriqué en début de notebook

In [ ]:
print(full_name)

In [ ]:
# LSTM, 5 FPS, Height = 60, Width = 80, nb_Classes = 10, 3 Epochs, Batch_size = 10, Pack_size = 50
if not model_charge: #On ne doit charger le modele que si il a été train cette fois-ci
    model.save(full_name)